In [ ]:
# Cell 1: Imports and Setup
# ==============================================================================
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

print("TensorFlow Version:", tf.__version__)

# Set a consistent style for plots
sns.set(style='whitegrid')

In [ ]:
# Cell 2: Data Loading and Preparation (CORRECTED)
# ==============================================================================
# Define the file paths
DATA_PATH = 'BGP_RIPE_datasets_for_anomaly_detection_csv_revised_19022021/'

# --- Load the BENIGN (normal) data ---
# We add header=None to tell pandas to use default integer column names.
print("Loading benign (normal) data...")
ripe_df = pd.read_csv(DATA_PATH + 'RIPE_regular.csv', header=None)
bcnet_df = pd.read_csv(DATA_PATH + 'BCNET_regular.csv', header=None)
normal_df = pd.concat([ripe_df, bcnet_df], ignore_index=True)

# --- Extract the 37 features for training ---
# Columns 4 to 40 will now have integer names. .iloc works on position, so it's correct.
normal_features = normal_df.iloc[:, 4:41]
print(f"Loaded {len(normal_features)} samples of normal data.")

# --- Load the ANOMALY (attack) data ---
print("\nLoading anomaly (attack) data...")
anomaly_files = [
    'WannaCrypt.csv', 'Moscow_blackout.csv', 'Slammer.csv', 'Nimda.csv', 'Code_Red_I.csv'
]
anomaly_dfs = {}
for file in anomaly_files:
    # Add header=None here as well for consistency.
    anomaly_dfs[file.split('.')[0]] = pd.read_csv(DATA_PATH + file, header=None)
    print(f"- Loaded {file}: {len(anomaly_dfs[file.split('.')[0]])} samples")

In [ ]:
# Cell 3: Data Scaling and Chronological Splitting
# ==============================================================================
# We must split the data chronologically BEFORE scaling to prevent data leakage.
# We will only train our model on normal data.

# Chronological split of the normal data
train_size = int(len(normal_features) * 0.8)
train_df = normal_features[:train_size]
val_df = normal_features[train_size:] # This part will be used to set the threshold

print(f"Training data size: {len(train_df)}")
print(f"Validation data size (for thresholding): {len(val_df)}")

# --- Scaling ---
# Initialize the scaler and fit it ONLY on the training data.
scaler = StandardScaler()
scaler = scaler.fit(train_df)

# Apply the scaler to our training and validation sets
train_scaled = scaler.transform(train_df)
val_scaled = scaler.transform(val_df)

In [ ]:
# Cell 4: Sequence Creation Function
# ==============================================================================
# This function converts our 2D data into 3D sequences for the LSTM.
def create_sequences(data, time_steps=60):
    sequences = []
    for i in range(len(data) - time_steps + 1):
        sequences.append(data[i:i + time_steps])
    return np.array(sequences)

# Define the lookback window (how many seconds of data the model sees at once)
TIME_STEPS = 60

# Create the sequences for training and validation
X_train_seq = create_sequences(train_scaled, TIME_STEPS)
X_val_seq = create_sequences(val_scaled, TIME_STEPS)

print(f"Training sequences shape: {X_train_seq.shape}")
print(f"Validation sequences shape: {X_val_seq.shape}")

In [ ]:
# Cell 5: Build the LSTM Autoencoder Model
# ==============================================================================
# Get the number of features from our data
n_features = X_train_seq.shape[2]

# Define the model architecture as described in the paper methodology
model = keras.Sequential([
    # Encoder
    keras.layers.Input(shape=(TIME_STEPS, n_features)),
    keras.layers.LSTM(128, activation='relu', return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(64, activation='relu', return_sequences=False),

    # Bridge
    keras.layers.RepeatVector(TIME_STEPS),

    # Decoder
    keras.layers.LSTM(64, activation='relu', return_sequences=True),
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(128, activation='relu', return_sequences=True),

    # Output Layer
    keras.layers.TimeDistributed(keras.layers.Dense(n_features))
])

# Compile the model
model.compile(optimizer='adam', loss='mae') # Mean Absolute Error is a good loss for reconstruction
model.summary()

In [ ]:
# Cell 6: Train the Model
# ==============================================================================
# Train the model to reconstruct normal sequences.
# We use a callback to stop training early if the model stops improving.
history = model.fit(
    X_train_seq, X_train_seq,  # Input and target are the same for an autoencoder
    epochs=30,                # A reasonable number of epochs
    batch_size=64,            # Adjust based on memory
    validation_data=(X_val_seq, X_val_seq),
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
    ],
    verbose=1
)

In [ ]:
# Cell 7: Determine the Anomaly Threshold (IMPROVED VERSION)
# ==============================================================================

# --- Part 1: Visualize the training loss (This part is unchanged) ---
plt.figure(figsize=(10, 5)) # Use a slightly wider figure for better readability
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training History')
plt.ylabel('Loss (Mean Absolute Error)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# --- Part 2: Calculate and visualize the threshold (This is the improved part) ---
# Get the reconstruction error on the validation (normal) data
print("\nCalculating reconstruction errors on the validation set...")
X_val_pred = model.predict(X_val_seq)
val_mae_loss = np.mean(np.abs(X_val_pred - X_val_seq), axis=(1, 2))

# Plot the distribution of reconstruction errors with annotations
print("Plotting the distribution of errors to determine threshold...")
plt.figure(figsize=(12, 6)) # A larger figure for the main plot
sns.histplot(val_mae_loss, bins=50, kde=True, stat="density")
plt.title('Distribution of Reconstruction Error on Normal Data (Validation Set)')
plt.xlabel('Mean Absolute Error (Reconstruction Error)')
plt.ylabel('Density')

# --- NEW: Add the threshold line and text to the plot ---
# Set the anomaly threshold at the 99th percentile of the validation loss
threshold = np.percentile(val_mae_loss, 99)
print(f"Anomaly Threshold (99th percentile) set to: {threshold:.4f}")

# Plot the threshold as a vertical red dashed line
plt.axvline(threshold, color='r', linestyle='--',
            label=f'99th Percentile Threshold = {threshold:.4f}')

# Add a legend to show what the red line means
plt.legend()
plt.show()

In [ ]:
# Cell 8: Final Evaluation on Anomaly and Normal Data (CORRECTED & More Robust)
# ==============================================================================

def evaluate_model(data, labels):
    """A helper function to scale, sequence, and evaluate data."""
    data_scaled = scaler.transform(data)
    sequences = create_sequences(data_scaled, TIME_STEPS)
    sequence_labels = labels[TIME_STEPS - 1:].to_numpy() # Convert to numpy array for clean indexing

    pred_sequences = model.predict(sequences)
    test_mae_loss = np.mean(np.abs(pred_sequences - sequences), axis=(1, 2))

    predicted_labels = (test_mae_loss > threshold).astype(int)

    return sequence_labels, predicted_labels

# --- Prepare the final, combined test set ---
# Start with the unseen normal data (the validation set)
# val_df already has the correct features (columns 4-40)
test_normal_features = val_df.copy()
# The label is 0 for all normal data.
test_normal_features['label'] = 0

# Now process and combine all anomaly files
all_test_dfs = [test_normal_features]
for name, df in anomaly_dfs.items():
    # df has columns 0-41. We want features from 4-40 and the label from 41.
    anomaly_features = df.iloc[:, 4:41].copy()
    # The original label is in column 41. Let's make it 1 for anomaly.
    anomaly_features['label'] = 1
    all_test_dfs.append(anomaly_features)

# Create one big test dataframe
final_test_df = pd.concat(all_test_dfs, ignore_index=True)

# Separate features and labels
final_test_labels = final_test_df['label']
# The feature columns are all columns EXCEPT the one named 'label'
final_test_features = final_test_df.drop('label', axis=1)


# --- Run the evaluation ---
true_labels, predicted_labels = evaluate_model(final_test_features, final_test_labels)

# --- Calculate and print metrics ---
print("Final Evaluation Metrics:")
print("="*25)
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# --- Plot the confusion matrix ---
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Benign', 'Anomaly'], yticklabels=['Benign', 'Anomaly'])
plt.title('Confusion Matrix')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Cell 9: Case Study - A Deeper Dive into a Specific Anomaly
# ==============================================================================

def run_case_study(anomaly_name, anomaly_df, scaler, model, threshold, time_steps):
    """
    A function to evaluate and visualize results for a single anomaly dataset.

    Args:
        anomaly_name (str): The name of the anomaly for titles and labels.
        anomaly_df (pd.DataFrame): The dataframe containing the anomaly data.
        scaler (StandardScaler): The pre-fitted scaler object.
        model (keras.Model): The trained LSTM autoencoder model.
        threshold (float): The anomaly detection threshold.
        time_steps (int): The sequence length.
    """
    print(f"--- Case Study: Evaluating {anomaly_name} Anomaly ---")

    # 1. Prepare data
    features = anomaly_df.iloc[:, 4:41]
    true_labels = np.ones(len(features)) # Ground truth is all anomalies (1)

    # 2. Scale and sequence
    scaled_features = scaler.transform(features)
    sequences = create_sequences(scaled_features, time_steps)
    true_labels_seq = true_labels[time_steps - 1:]

    # 3. Get predictions and reconstruction error
    print(f"Predicting on {anomaly_name} sequences...")
    pred_sequences = model.predict(sequences)
    mae_loss = np.mean(np.abs(pred_sequences - sequences), axis=(1, 2))

    # 4. Classify and calculate recall
    predicted_labels = (mae_loss > threshold).astype(int)
    recall = recall_score(true_labels_seq, predicted_labels)

    print(f"\nDetection Recall for {anomaly_name}: {recall:.4f}")
    print(f"Detected {np.sum(predicted_labels)} out of {len(true_labels_seq)} anomalous sequences as anomalies.")

    # 5. Visualize the results
    print(f"\nVisualizing reconstruction errors for {anomaly_name} vs. Normal Data...")
    plt.figure(figsize=(12, 6))

    # Plot the distribution of normal errors (val_mae_loss should be available from Cell 7)
    sns.histplot(val_mae_loss, bins=50, kde=True, stat="density", color="blue", label='Normal Data Error')
    # Plot the distribution of the current anomaly's errors
    sns.histplot(mae_loss, bins=50, kde=True, stat="density", color="red", label=f'{anomaly_name} Data Error')
    # Plot the threshold line
    plt.axvline(threshold, color='black', linestyle='--', label=f'Anomaly Threshold = {threshold:.4f}')

    plt.title(f'Comparison of Reconstruction Error: Normal vs. {anomaly_name}')
    plt.xlabel('Mean Absolute Error (Reconstruction Error)')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

# --- Now, let's run the case study for WannaCry ---
# We retrieve the dataframe from the dictionary created in Cell 2.
# Ensure Cell 2 has been run!
wannacry_df_to_test = anomaly_dfs['WannaCrypt']
run_case_study('WannaCry', wannacry_df_to_test, scaler, model, threshold, TIME_STEPS)

# --- You can easily run it for another attack too! ---
# slammer_df_to_test = anomaly_dfs['Slammer']
# run_case_study('Slammer', slammer_df_to_test, scaler, model, threshold, TIME_STEPS)

In [ ]:
# Cell 10: Case Study - Explicitly Testing the Moscow Blackout Anomaly
# ==============================================================================

# --- This section runs the case study specifically for the Moscow blackout ---
# The 'run_case_study' function was defined in Cell 9. We are just calling it again
# with a different dataset.

# First, ensure the 'anomaly_dfs' dictionary is available from Cell 2.
if 'anomaly_dfs' in locals() and 'Moscow_blackout' in anomaly_dfs:

    # Retrieve the Moscow blackout dataframe from the dictionary
    moscow_df_to_test = anomaly_dfs['Moscow_blackout']

    # Call the main function to perform the evaluation and visualization
    run_case_study(
        anomaly_name='Moscow Blackout',
        anomaly_df=moscow_df_to_test,
        scaler=scaler,
        model=model,
        threshold=threshold,
        time_steps=TIME_STEPS
    )

else:
    print("Error: The 'anomaly_dfs' dictionary is not defined or does not contain 'Moscow_blackout'.")
    print("Please make sure you have run Cell 2 of the notebook successfully.")

In [ ]:
# Cell 11: Case Study - Explicitly Testing the Slammer Worm Anomaly
# ==============================================================================

# --- This section runs the case study specifically for the Slammer worm ---
# We will reuse the 'run_case_study' function defined in Cell 9.

# First, ensure the 'anomaly_dfs' dictionary is available from Cell 2.
if 'anomaly_dfs' in locals() and 'Slammer' in anomaly_dfs:

    # Retrieve the Slammer dataframe from the dictionary
    slammer_df_to_test = anomaly_dfs['Slammer']

    # Call the main function to perform the evaluation and visualization
    run_case_study(
        anomaly_name='Slammer Worm',
        anomaly_df=slammer_df_to_test,
        scaler=scaler,
        model=model,
        threshold=threshold,
        time_steps=TIME_STEPS
    )

else:
    print("Error: The 'anomaly_dfs' dictionary is not defined or does not contain 'Slammer'.")
    print("Please make sure you have run Cell 2 of the notebook successfully.")

In [ ]:
# Cell 12: Generate a Synthetic "BGP Storm" Anomaly Dataset
# ==============================================================================
import numpy as np
import pandas as pd

print("Generating a synthetic anomaly dataset: 'BGP_Storm_Anomaly.csv'")

# --- Define Parameters for our Synthetic Anomaly ---
num_samples = 300  # A 5-minute anomaly event (300 seconds)
num_features = 37

# --- Create a base matrix of near-zero values ---
# This simulates a baseline before the storm hits hard.
data = np.random.uniform(low=0.0, high=0.1, size=(num_samples, num_features))

# --- Inject the "BGP Storm" characteristics into key features ---
# The storm will run for the middle 200 seconds of the event.
start_index = 50
end_index = 250
storm_duration = end_index - start_index

# Feature 1 (col 0): Number of announcements - Make it high and erratic
data[start_index:end_index, 0] = np.random.randint(800, 1500, size=storm_duration)

# Feature 2 (col 1): Number of withdrawals - Also high and erratic
data[start_index:end_index, 1] = np.random.randint(800, 1500, size=storm_duration)

# Feature 5 (col 4): Average AS-path length - Make it abnormally long
data[start_index:end_index, 4] = np.random.randint(15, 25, size=storm_duration)

# Feature 11 (col 10): Average edit distance - THIS IS A KEY INDICATOR OF CHAOS
# Make it consistently high, showing constant path changes.
data[start_index:end_index, 10] = np.random.uniform(8.0, 15.0, size=storm_duration)

# Feature 12 (col 11): Maximum edit distance - Also very high
data[start_index:end_index, 11] = data[start_index:end_index, 10] + np.random.randint(2, 5, size=storm_duration)

# Feature 37 (col 36): Packet size - Make it large and variable due to long AS-paths
data[start_index:end_index, 36] = np.random.randint(2000, 5000, size=storm_duration)


# --- Assemble the final DataFrame in the correct format ---
# Create a DataFrame from our numpy array
features_df = pd.DataFrame(data)

# Create dummy time columns (not used by the model, but needed for format consistency)
time_df = pd.DataFrame(np.zeros((num_samples, 4)))

# Create the label column - all are anomalies (label=1)
labels_df = pd.DataFrame(np.ones((num_samples, 1)))

# Concatenate all parts together
final_synthetic_df = pd.concat([time_df, features_df, labels_df], axis=1)

# --- Save to a CSV file ---
# Save without a header and without the pandas index
file_path = 'BGP_Storm_Anomaly.csv'
final_synthetic_df.to_csv(file_path, header=False, index=False)

print(f"Successfully created '{file_path}' with {num_samples} samples.")
print("You can now upload this file or use it in the next cell to test the model.")

In [ ]:
# Cell 13: Case Study - Testing the Synthetic "BGP Storm" Anomaly
# ==============================================================================

# --- This section runs the case study on our generated anomaly data ---

# Define the name and load the dataframe we just created
anomaly_name_to_test = 'BGP Storm'
try:
    anomaly_df_to_test = pd.read_csv('BGP_Storm_Anomaly.csv', header=None)

    # Call the main function to perform the evaluation and visualization
    run_case_study(
        anomaly_name=anomaly_name_to_test,
        anomaly_df=anomaly_df_to_test,
        scaler=scaler,
        model=model,
        threshold=threshold,
        time_steps=TIME_STEPS
    )

except FileNotFoundError:
    print("Error: 'BGP_Storm_Anomaly.csv' not found.")
    print("Please make sure you have run the generation script in the previous cell successfully.")